In [2]:
import numpy as np
import torch
import torch.nn as nn
import data_loader
import torch.optim as opt
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook as tqdm
from model import MF

In [39]:
class MUD(nn.Module):
    """
        - userLen: the number of users
        - itemLen: the number of items
        - params: the parameters dict used for constructing model
            - l_size: latent dimension size
            - gpu: True/False, whether using GPU
            
    """
    def __init__(self, userLen, itemLen, params):
        super(MUD, self).__init__()
        self.userNum = userLen
        self.itemNum = itemLen
        self.params = params
        if 'gpu' in params and params['gpu'] == True:
            self.device = 'cuda'
        else:
            self.device = 'cpu'

        l_size = params['l_size']
        
        """
            Initialize  global bias,
                        user bias,
                        item bias,
                        user embedding,
                        item embedding
        """
        self.globalBias = nn.Embedding(1,1)
        self.uBias = nn.Embedding(userLen,1)
        self.itemBias = nn.Embedding(itemLen,1)
        self.uEmbed = nn.Embedding(userLen, l_size)
        self.itemEmbed = nn.Embedding(itemLen, l_size)
    
    def forward(self, users, items, price, rating):
        gB = self.globalBias.weight.data.expand(users.shape[0],1)
        uE = self.uEmbed(users)
        uB = self.uBias(users)
        iE = self.itemEmbed(items)
        iB = self.itemBias(items)
        alpha = gB + uB + iB + torch.mul(uE, iE).sum(1).view(-1,1) 
        mud =  torch.mul(torch.tanh(rating),rating)/(2*torch.sigmoid(price))
        return mud
        

In [35]:
params = dict()
params["lr"] = 0.01
params["lr_decay"] = 0.001
params["batch_size"] = 512
params["epoch_limit"] = 10
params["w_decay"] = 0.9
params["momentum"] = 0.0
params["n_neg"] = 1
params["epsilon"] = 1e-4
# model parameters
params["l_size"] = 32
params["gpu"] = False

In [36]:
train, val, test = data_loader.read_data("Baby")
price = data_loader.get_price("Baby")
related = data_loader.read_related("Baby")
trainset = data_loader.TransactionData(train,related,price)

In [40]:
model = MUD(userLen = trainset.userNum, itemLen = trainset.itemNum, params = params)

# optimizer: weight_decay correspond to L2 penalty
optimizer = opt.SGD(model.parameters(), lr = params["lr"], \
                    weight_decay = params["w_decay"], momentum = params["momentum"])

# objectives: pairwise loss
criterion = nn.LogSoftmax()

# data loaders: train, val, test

trainset.set_negN(params["n_neg"])
trainLoader = DataLoader(trainset, batch_size = params["batch_size"], shuffle = True, num_workers = 0)

In [42]:
epsilon = params["epsilon"]

epoch = 0
error = np.float("inf")
trainErrorList = []
valErrorList = []
valHistory = []
explodeTempreture = 3
convergenceTempreture = 3

while epoch < params["epoch_limit"]:
    epoch = epoch + 1
    
    # epoch training
    
    print("Epoch " + str(epoch) + " training...")
    L = len(trainLoader.dataset)
    pbar = tqdm(total = L)
    runningLoss = list()
    model.train()
    for i, batchData in enumerate(trainLoader):
        
        # batch training
        
        optimizer.zero_grad()
        
        # get input
        users = torch.LongTensor(batchData['user']).to(model.device) # users
        items = torch.LongTensor(batchData['item']).to(model.device) # positive samples
        posPrice = torch.FloatTensor(batchData['price']).to(model.device)
        posRating = torch.FloatTensor(batchData['rating']).to(model.device)

        
        npUsers = users.view(-1,1) 
        npUsers = npUsers.expand(npUsers.shape[0], params["n_neg"]) # users expanded to match negative samples
        negItems = torch.LongTensor(batchData['negItem']).to(model.device) # negative samples, default #=4
        negPrice = torch.FloatTensor(batchData['negPrice']).to(model.device)
        posRating = torch.FloatTensor(batchData['negRating']).to(model.device)
        
        # forward
        pOut = model.forward(users, items, posPrice, posRating) # ranking score of positive samples
        pOut = pOut.view(-1,1)
        pOut = pOut.expand(pOut.shape[0], params["n_neg"]) # results are expanded to match negative samples
        pOut = pOut.reshape(-1)
        
        nOut = model.forward(npUsers.reshape(-1), negItems.reshape(-1)) # ranking score of negative samples

        # loss
        loss = criterion(pOut, nOut, torch.ones_like(nOut).to(model.device)) # pairwise loss
        runningLoss.append(loss.item())
        # backward
        loss.backward()
        # optimize
        optimizer.step()
        
        # progress report
        pbar.update(users.shape[0])
        # running loss update
        if len(runningLoss) >= 50:
            trainErrorList.append(np.mean(runningLoss))
            runningLoss = list()

Epoch 1 training...


HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

Epoch 2 training...


HBox(children=(IntProgress(value=0, max=132748), HTML(value='')))

KeyboardInterrupt: 